# Exploratory Data Analysis: Lichess database

This jupyter notebook contains an exploratory data analysis done over a chess database with information from 200k games. Each game (the rows of the data frame) contains categorical variates, such as the type of opening (ECO), and numerical variates, such as the elo of each player.

(Tentative) The main objective of this EDA was to determine regularities in the playstyle and how these changed when variates, such as elo, changed. Furthermore, the analysis includes general statistics calculations including location, scale, and shape parameters (mean value, variance, skewness, and kurtosis) which helped draw conclusions from a set of hypothesis tests.

### Remarks

1. In general, the notebook has the following structure: markdown cell --> code cells. 

2. Each code cell includes hashed text explaining briefly what the code is supposed to be doing. Additional commentary is contained in the markdown cells.

3. All the required libraries are included in the first code cell, namely the preamble of the notebook. Exceptions include particular ```scipy``` submodules used for miscellaneous statistics calculations.

4. Recurrent code is included in the ```functions.py``` module.

5. All the pertinent references are in the ```README.md``` file.

6. **All the graphs** contained in this notebook are saved in the ```graphs``` directory.

7. The dataset is available in the github repository as ```modified_dataset```.

In [32]:
# Preamble
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import functions as f

In [33]:
# Loading dataset
games_df = pd.read_csv('modified_dataset', delimiter = ',')
games_df.drop('index', inplace = True, axis = 1)
games_df

,BlackElo,BlackRatingDiff,Date,ECO,Result,Termination,UTCTime,WhiteElo,WhiteRatingDiff,BlackTitle,WhiteTitle,Category,Weekday,Movements
0,906,13.0,2019.04.30,B15,0-1,Normal,22:00:24,971,-12.0,NaN,NaN,Blitz,Tuesday,73
1,1296,28.0,2019.04.30,C50,0-1,Normal,22:00:13,1312,-10.0,NaN,NaN,Blitz,Tuesday,67
2,1761,-13.0,2019.04.30,C41,1-0,Normal,22:00:41,1653,27.0,NaN,NaN,Rapid,Tuesday,71
3,2404,8.0,2019.04.30,B06,0-1,Normal,22:00:43,2324,-8.0,NaN,FM,Bullet,Tuesday,85
4,1595,-10.0,2019.04.30,B32,1-0,Normal,22:00:46,1614,29.0,NaN,NaN,Blitz,Tuesday,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,1491,10.0,2019.05.22,B00,0-1,Normal,21:17:00,1456,-9.0,NaN,NaN,Rapid,Wednesday,30
199996,1549,-10.0,2019.05.22,C21,1-0,Time forfeit,21:17:09,1575,25.0,NaN,NaN,Classical,Wednesday,59
199997,1450,-11.0,2019.05.22,D20,1-0,Normal,21:17:09,1439,11.0,NaN,NaN,Blitz,Wednesday,42
199998,1588,40.0,2019.05.22,B07,0-1,Normal,21:17:13,1753,-18.0,NaN,NaN,Bullet,Wednesday,45


## Goodness of fit test for Elo distribution

$H_0$: Average Elo data is beta distributed.

$H_1$: Average Elo data deviaties from a beta distribution.

In [34]:
from scipy import stats
from functools import partial

avg_elo = (games_df['WhiteElo'] + games_df['BlackElo'])/2 # Calculates average elo of each game
data = avg_elo
beta_param_bounds = [(4,5),(7,9),(600, 700),(2400,2500)] # Prepares bounds for the a, b, loc, and scale parameters of the beta fit.
fit_result = stats.fit(stats.beta, data, beta_param_bounds) # Gives results of the fit
a, b, loc, scale = fit_result.params 
beta_fit = stats.beta(a, b, loc, scale) # Distribution that fits the data best

In [35]:
# Kolmogorov - Smirnov test
D, p_value = stats.kstest(data, partial(stats.beta.cdf, a = a, b = b, loc = loc, scale = scale))
print('Maximum function difference (KS test statistic): {}'.format(D))
print('p value: {}'.format(p_value))
alpha = 0.05
if p_value < alpha:
    print('The null hypothesis is rejected.')
else:
    print('The null hypothesis is not rejected.')

Maximum function difference (KS test statistic): 0.019851123764107748
p value: 6.806008160762547e-69
The null hypothesis is rejected.


In [36]:
fig, axes = plt.subplots(1, 2, figsize = (12, 3)) # Definition of graphing instances

x = np.linspace(0, 3000, 10000) # Graphing range

pdf = beta_fit.pdf(x) # Probability density function
cdf = beta_fit.cdf(x) # Cumulative distribution function

axes[0].plot(x, pdf, color = 'r', label = 'Fit') # Graphs PDF
sns.histplot(data, ax = axes[0], stat = 'density',binwidth = 50, label = 'Data') # Graphs histogram of data

N = 2e5 # Number of games
sorted_data = np.sort(data) # Ordered data
edf = np.arange(N) / float(N) # 'Experimental' cumulative distribution function

axes[1].plot(x, cdf, color = 'r', label = 'CDF') # Graphs CDF
axes[1].plot(sorted_data, edf, label = 'EDF') # Graphs EDF

for ax in axes.flat:
    ax.set(xlabel = 'Average Elo')

axes[1].set(ylabel = 'Cumulative probability')
plt.legend()
fig.savefig('graphs/kstest-avg_elo', dpi = 700, format = 'pdf', bbox_inches = 'tight')

## ANOVA test: Win rate and opening

Each game is grouped in one of five categories according to the first letter of ECO.

$H_0$: The mean win rate for each of the groups is the same.

$H_1$: At least one group has a different mean win rate.

## ANOVA test: Win rate and time of the day

Each game is grouped in one of ??? groups according to UTCTime.

$H_0$: The mean win rate is the same at any time of the day.

$H_1$: The mean win rate is higher for a certain time of the day.

## ANOVA test: Win rate and Elo range

Each game is classified in one of 4 tiers of elo according to the mean elo between black and white: low elo, mid elo, high elo, very high elo.

$H_0$: The mean win rate is the same for each elo tier.
$H_1$: The mean win rate of at least one tier is different.

In [49]:
import scipy.stats
from scipy.stats import chi2_contingency

games_df['EloTier'] = ['Very high elo' if elo >= 2000 else 'High elo' if elo >= 1800 else \
    'Mid elo' if elo >= 1400 else 'Low elo' for elo in avg_elo] #Create a new EloTier column
games_df['Result'].replace({'0-1': 'Lose', '1/2-1/2': 'Draw', '*': 'Abandoned', '1-0': 'Win'}, inplace = True) #Replace the numeric with categorical variables
#games_df['EloTier'].replace({'Very high elo': np.NAN, 'Low elo': np.NAN}, inplace = True) #'Very high elo' and 'Low elo' are considered outliers
games_df

,BlackElo,BlackRatingDiff,Date,ECO,Result,Termination,UTCTime,WhiteElo,WhiteRatingDiff,BlackTitle,WhiteTitle,Category,Weekday,Movements,EloTier
0,906,13.0,2019.04.30,B15,Lose,Normal,22:00:24,971,-12.0,NaN,NaN,Blitz,Tuesday,73,Low elo
1,1296,28.0,2019.04.30,C50,Lose,Normal,22:00:13,1312,-10.0,NaN,NaN,Blitz,Tuesday,67,Low elo
2,1761,-13.0,2019.04.30,C41,Win,Normal,22:00:41,1653,27.0,NaN,NaN,Rapid,Tuesday,71,Mid elo
3,2404,8.0,2019.04.30,B06,Lose,Normal,22:00:43,2324,-8.0,NaN,FM,Bullet,Tuesday,85,Very high elo
4,1595,-10.0,2019.04.30,B32,Win,Normal,22:00:46,1614,29.0,NaN,NaN,Blitz,Tuesday,71,Mid elo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,1491,10.0,2019.05.22,B00,Lose,Normal,21:17:00,1456,-9.0,NaN,NaN,Rapid,Wednesday,30,Mid elo
199996,1549,-10.0,2019.05.22,C21,Win,Time forfeit,21:17:09,1575,25.0,NaN,NaN,Classical,Wednesday,59,Mid elo
199997,1450,-11.0,2019.05.22,D20,Win,Normal,21:17:09,1439,11.0,NaN,NaN,Blitz,Wednesday,42,Mid elo
199998,1588,40.0,2019.05.22,B07,Lose,Normal,21:17:13,1753,-18.0,NaN,NaN,Bullet,Wednesday,45,Mid elo


In [50]:
contingency_games = pd.crosstab(games_df["EloTier"], games_df["Result"]) #contingency table
contingency_games


Result,Abandoned,Draw,Lose,Win
EloTier,,,,
High elo,0,903,11356,12248
Low elo,0,1341,36022,37623
Mid elo,0,2247,41227,44080
Very high elo,5,654,5894,6400


In [51]:
#chi_2 test
chi2_stat, p, dof, expected = scipy.stats.chi2_contingency(contingency_games)
print(f"chi2 statistic:     {chi2_stat:.5g}")
print(f"p-value:            {p:.5g}")
print(f"degrees of freedom: {dof}")
print("expected frequencies:\n",expected)


chi2 statistic:     703.36
p-value:            1.3134e-145
degrees of freedom: 9
expected frequencies:
 [[6.12675000e-01 6.30442575e+02 1.15794350e+04 1.22965098e+04]
 [1.87465000e+00 1.92901485e+03 3.54305101e+04 3.76246004e+04]
 [2.18885000e+00 2.25232665e+03 4.13688272e+04 4.39306573e+04]
 [3.23825000e-01 3.33215925e+02 6.12022774e+03 6.49923251e+03]]


## Is there correlation between Termination and RatingDiff?

## Is there correlation between Result and RatingDiff?